This script test the number of pathways and nodes present after we subset the 83 severe samples to 45 to match the mild cases. We run this 100 times. Using the same subset dataset we test for the whole dataset, and also the severe naive networks.

We make two different networks, one for the COVID cases 1-2 compared to COVID cases 3-7 <br>
This is because there are only 18 samples in common between the metabolomic and proteomic datasets

0       Common samples: 18           Metabolomic samples: 133        Proteomic samples: 123 <br>
1-2       Common samples: 45          Metabolomic samples: 45        Proteomic samples: 48 <br>
3-4       Common samples: 56          Metabolomic samples: 57        Proteomic samples: 59 <br>
5-7       Common samples: 27          Metabolomic samples: 28        Proteomic samples: 28 <br>

146 common samples overall,   128 cases, composed of (45 samples (WHO 1-2) vs 83 samples (WHO 3-7))

### Reading in the files

In [51]:
import pandas as pd
import sspa
import random
from sklearn.preprocessing import StandardScaler

In [2]:
#Load the common cases dataset
df = pd.read_csv('Data/Su_COVID_metabolomics_processed_commoncases.csv', index_col=0)
reactome_pathways = sspa.process_gmt("Data/Reactome_Homo_sapiens_pathways_compounds_R84.gmt")

### Subset the severe case samples and adding back onto the mild cases

In [3]:
df_mild = (df[df["WHO_status"] == '1-2']) #45 samples, no need to remove the metadata, since I do that in a later step
df_severe = (df[(df["WHO_status"] == '3-4') | (df["WHO_status"] == '5-7')]) #83 samples

In [4]:
df_severe

,1372,16610,72665,27823,30915,37373,16831,545959,17050,16359,...,133693,133694,28036,28238,76341,89312,17861,89188,WHO_status,Group
sample_id,,,,,,,,,,,,,,,,,,,,,
INCOV001,-0.342522,-0.830841,-0.703451,-0.451278,0.153207,-0.443252,-0.228725,0.501998,-0.861886,-0.344155,...,-0.465505,0.055753,-0.225622,0.400398,0.163210,1.579262,-0.162221,-0.603790,3-4,COVID19
INCOV002,-0.125585,-1.002246,0.399273,-0.828341,-0.980576,0.373921,-1.076979,-0.131876,-0.869093,-0.467326,...,0.509336,0.627269,0.321718,-0.158514,0.001469,-1.051485,1.299330,0.578560,5-7,COVID19
INCOV003,-0.859326,-0.409159,3.573100,-0.546783,-0.993647,1.547649,0.370131,-0.020399,-0.703786,-0.015627,...,2.056610,1.717149,-0.297723,-1.276891,-0.658288,2.141594,-1.091183,0.447197,5-7,COVID19
INCOV004,-0.562999,-1.560249,-1.269383,-1.437919,0.108734,-0.192066,0.161591,-0.586759,-1.457246,-0.500612,...,0.793065,-0.033125,-0.366082,-0.642671,1.724436,2.074193,-1.021184,-0.410849,3-4,COVID19
INCOV005,-0.741957,0.387909,-0.711036,-0.711515,0.670720,0.610591,-0.322241,-0.793451,-1.230681,-0.246493,...,-0.184967,-0.469553,-0.293869,0.558366,-0.390308,-0.455737,-0.535223,-0.847727,3-4,COVID19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INCOV133,0.106495,0.944466,-0.550708,-0.124598,0.115790,0.637512,0.464733,0.280852,0.545110,-0.489136,...,-0.066113,0.582042,-0.570565,-0.792820,0.092206,-0.492807,1.157865,-0.752052,3-4,COVID19
INCOV136,-0.174602,1.167644,0.055499,0.113342,-0.663851,0.799367,0.004697,0.748894,1.328753,0.391221,...,-0.741578,-0.730981,-0.380517,-1.251641,0.389163,-0.603048,0.941619,-0.511694,3-4,COVID19
INCOV137,0.274900,-0.014841,-0.928512,0.885263,-1.591862,0.230250,0.057818,-0.775962,-0.179700,-0.534786,...,-1.306132,-1.246428,1.763824,1.258387,0.448359,-0.508931,0.268474,-0.997206,5-7,COVID19


In [43]:
sample_list = []
while len(sample_list) < 47:
    subset = random.choice(df_severe.index.tolist())
    if subset not in sample_list:
        sample_list.append(subset)

In [44]:
len(sample_list)

47

In [47]:
#Subsetting the severe df to the sample list only
new_df_severe = df_severe.loc[df_severe.index.isin(sample_list), : ]
new_df_severe

,1372,16610,72665,27823,30915,37373,16831,545959,17050,16359,...,133693,133694,28036,28238,76341,89312,17861,89188,WHO_status,Group
sample_id,,,,,,,,,,,,,,,,,,,,,
INCOV001,-0.342522,-0.830841,-0.703451,-0.451278,0.153207,-0.443252,-0.228725,0.501998,-0.861886,-0.344155,...,-0.465505,0.055753,-0.225622,0.400398,0.163210,1.579262,-0.162221,-0.603790,3-4,COVID19
INCOV002,-0.125585,-1.002246,0.399273,-0.828341,-0.980576,0.373921,-1.076979,-0.131876,-0.869093,-0.467326,...,0.509336,0.627269,0.321718,-0.158514,0.001469,-1.051485,1.299330,0.578560,5-7,COVID19
INCOV003,-0.859326,-0.409159,3.573100,-0.546783,-0.993647,1.547649,0.370131,-0.020399,-0.703786,-0.015627,...,2.056610,1.717149,-0.297723,-1.276891,-0.658288,2.141594,-1.091183,0.447197,5-7,COVID19
INCOV006,1.273480,-1.071737,-0.805960,-1.133776,0.665464,1.104179,-1.229347,-0.283827,-1.003506,-0.177539,...,-0.533101,-0.311120,-1.531571,-0.355830,-0.993789,-0.188828,0.971550,1.431262,3-4,COVID19
INCOV007,-0.136587,-0.393119,-0.294327,-0.628552,-0.177814,-0.260732,-0.525029,-0.717994,-0.692478,-0.106265,...,-0.309071,-0.101201,-0.327181,-0.566064,0.017056,0.091596,0.654008,-0.327013,3-4,COVID19
INCOV013,-0.424609,-0.758431,0.520350,-1.197386,-0.364411,0.898000,0.085133,-0.665907,-1.388233,-0.446587,...,0.113726,0.152841,-0.550163,-1.579064,1.076975,-0.511027,-0.971738,0.346868,5-7,COVID19
INCOV015,-1.012235,-0.956403,-0.405500,-0.930118,3.690804,-0.831973,-0.429826,0.143811,-0.712541,0.928507,...,0.092085,-0.170113,-0.269837,-0.661984,-0.816174,-0.605692,0.857053,-0.886883,3-4,COVID19
INCOV016,-0.628094,-1.172915,0.352961,-0.875940,0.409128,2.115760,-0.358290,0.004944,-1.104045,0.384876,...,0.075013,-0.290120,-0.473562,1.198909,-0.092835,0.036063,0.483396,-1.218819,3-4,COVID19
INCOV018,0.576373,-0.615139,-0.069942,-0.711180,3.787851,1.191754,0.094155,-0.573700,-1.224146,0.310379,...,-0.713934,-0.395035,0.026235,2.657141,-0.713727,-0.657965,-0.153286,-0.900097,3-4,COVID19


In [48]:
df = pd.concat([df_mild, df_severe], axis=0)

### Scaling the data

In [ ]:
#df_norm = pd.DataFrame(StandardScaler().fit_transform(df_num),columns=df_num.columns, index=df_num.index)

#Add metadata to the end of the df
#df_final = pd.concat([df_norm, df.iloc[:,-2:]],axis=1) 
#df_final.to_csv('Data/Su_COVID_metabolomics_processed_commoncases.csv')

In [52]:
df_num  = df.iloc[:,:-2] #all rows, all columns apart from last two
df_norm = pd.DataFrame(StandardScaler().fit_transform(df_num),columns=df_num.columns, index=df_num.index)

In [53]:
df_final = pd.concat([df_norm, df.iloc[:,-2:]],axis=1) #add metadata back on

### Pathway analysis

### Network analysis

### Writing the results